In [3]:
%load_ext autoreload
%autoreload 2

In [16]:
from library import datasets, models
from flax import linen as nn
from jax import random, numpy as jnp
import optax
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

In [5]:
LATENT_DIM = 1
AMBIENT_DIM = 2

In [6]:
taylor_pair_dis = datasets.point_dataset.PointPairDistribution.random_taylor(
    latent_dim=LATENT_DIM,
    dis_A_dim=AMBIENT_DIM,
    dis_B_dim=AMBIENT_DIM,
    latent_range=1,
    max_order=2,
    coeff_range=1,
    noise_std_A=0.05,
    noise_std_B=0,
    key=random.PRNGKey(2))

dataset = taylor_pair_dis.generate_dataset(1000)

In [7]:
A, _ = dataset.get_all_point_pairs()

In [91]:
discriminator = nn.Sequential([
    nn.Dense(8),
    nn.relu,
    nn.Dense(4),
    nn.relu,
    nn.Dense(1),
])

generator = nn.Sequential([
    nn.Dense(8),
    nn.relu,
    nn.Dense(8),
    nn.relu,
    nn.Dense(4),
    nn.relu,
    nn.Dense(AMBIENT_DIM),
])

model = models.vanilla_gan.VanillaGAN(
    generator=generator,
    discriminator=discriminator,
    latent_shape=(LATENT_DIM,),
    ambient_shape=(AMBIENT_DIM,)
)

print(model)
model.initialize(loss_fn=optax.sigmoid_binary_cross_entropy)

generator:
	Model Structure:
		Sequential(
		    # attributes
		    layers = [Dense(
		        # attributes
		        features = 8
		        use_bias = True
		        dtype = None
		        param_dtype = float32
		        precision = None
		        kernel_init = init
		        bias_init = zeros
		    ), <jax._src.custom_derivatives.custom_jvp object at 0x7f3099d79300>, Dense(
		        # attributes
		        features = 8
		        use_bias = True
		        dtype = None
		        param_dtype = float32
		        precision = None
		        kernel_init = init
		        bias_init = zeros
		    ), <jax._src.custom_derivatives.custom_jvp object at 0x7f3099d79300>, Dense(
		        # attributes
		        features = 4
		        use_bias = True
		        dtype = None
		        param_dtype = float32
		        precision = None
		        kernel_init = init
		        bias_init = zeros
		    ), <jax._src.custom_derivatives.custom_jvp object at 0x7f3099d79300>, Dense(
		        # attributes
		        

In [112]:
model.train(A,optax.sgd(learning_rate=1e-2), print_every=5, batch_size=64, num_epochs=10)

iteration 14; gen_loss: -0.02252056449651718; dis_loss: 0.7031190395355225: : 15it [00:02,  6.51it/s]
iteration 14; gen_loss: -0.023081358522176743; dis_loss: 0.6931673288345337: : 15it [00:02,  6.81it/s]
iteration 14; gen_loss: -0.02557365596294403; dis_loss: 0.6938067674636841: : 15it [00:02,  6.88it/s]
iteration 14; gen_loss: -0.025932222604751587; dis_loss: 0.6950560808181763: : 15it [00:02,  6.97it/s]
iteration 14; gen_loss: -0.025054551661014557; dis_loss: 0.693142294883728: : 15it [00:02,  6.91it/s]
iteration 14; gen_loss: -0.0267430879175663; dis_loss: 0.6894301176071167: : 15it [00:02,  6.79it/s]
iteration 14; gen_loss: -0.025200633332133293; dis_loss: 0.6835439205169678: : 15it [00:02,  6.72it/s]
iteration 14; gen_loss: -0.02975919470191002; dis_loss: 0.6825281381607056: : 15it [00:02,  6.98it/s]
iteration 14; gen_loss: -0.03510475158691406; dis_loss: 0.6839035749435425: : 15it [00:02,  6.81it/s]
iteration 14; gen_loss: -0.036526814103126526; dis_loss: 0.6835451126098633: : 1

In [113]:
gan_distribution = model.create_distribution()

In [114]:
true_samples = A

In [115]:
fake_samples = gan_distribution.draw_samples(random.normal(random.PRNGKey(0), (1000, LATENT_DIM)))

In [116]:
print(true_samples.shape, fake_samples.shape)

(1000, 2) (1000, 2)


In [117]:
df = {'x': jnp.concatenate((true_samples[:, 0], fake_samples[:, 0]), axis=0),
      'y': jnp.concatenate((true_samples[:, 1], fake_samples[:, 1]), axis=0),
      'labels': jnp.concatenate((jnp.ones((true_samples.shape[0],)), jnp.zeros((fake_samples.shape[0],))), axis=0)}

In [118]:
px.scatter(df, x='x', y='y', color='labels')